### ИНСТРУКЦИИ И РЕКОМЕНДАЦИИ ДЛЯ СОЗДАНИЯ БАЗЫ ДАННЫХ ПРОТЕОМОВ

База данных для филогенетического исследования должна содержать:
- базу данных BLAST с выбранными протеомами;
- файл, приводящий гены и белки в соответствие друг другу (g2r.tsv);
- файл, приводящий идентификатор каждого таксона с его научным названием (names.dmp).

Мы собираемся получить наиболее полные протеомы из базы данных RefSeq, поэтому, если вы знаете, что будете использовать свой собственный набор протеомов, пожалуйста, соответствующим образом измените алгоритм. Кроме того, нам понадобятся утилиты BLAST+, а также Entrez Direct.

Обратите внимание, что алгоритм не предназначен для изучения генов прокариот, поскольку в этих протеомах часто используются неизбыточные номера присоединения белков (с префиксом 'WP_').

#### 1. Получение необходимых файлов.

Сначала загрузите текущий список протеомов из RefSeq с соответствующими оценками полноты BUSCO (вам потребуется установленный Entrez Direct).

In [ ]:
%%bash

mkdir refseq_proteomes
cd refseq_proteomes
esearch -db assembly -query 'has_egap_annotation[prop] AND "latest refseq"[filter]' \
  | esummary \
  | xtract \
    -pattern DocumentSummary \
    -element \
      AssemblyAccession,\
      Organism,\
      Taxid,\
      Busco/BuscoLineage,\
      Busco/TotalCount,\
      Busco/Complete,\
      Busco/SingleCopy,\
      Busco/Duplicated,\
      Busco/Fragmented,\
      Busco/Missing \
> busco_scores.tsv

Затем, получим файл taxdump.tar.gz.

In [ ]:
%%bash

cd refseq_proteomes
mkdir taxonomy
cd taxonomy
wget -q ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump.tar.gz -O taxdump.tar.gz 
sleep 1
tar -xzf taxdump.tar.gz
rm taxdump.tar.gz
rm citations.dmp
rm delnodes.dmp
rm division.dmp
rm gencode.dmp
rm images.dmp
rm merged.dmp
rm readme.txt
rm gc.prt

#### 2. Обнаружение самых полных протеомов.

Нам не нужно включать в анализ все протеомы: достаточно взять несколько наиболее полных протеомов из каждого класса или другого таксономического ранга.

Возьмём три самых полных протеома в каждом классе.

Загрузим nodes.dmp и names.dmp для работы с таксонами:

In [ ]:
import pandas as pd

sep = '\t\|\t'

nodes = pd.read_table(
    'refseq_proteomes/taxonomy/nodes.dmp',
    sep = sep,
    header = None,
    engine ='python'
)

nodes.columns = [
	'Taxid',
 	'Parent',
 	'Rank',
 	'EMBL',
 	'Division',
 	'Inherited_div',
 	'Gencode',
 	'Inherited_gencode',
 	'Mito',
 	'Inherited_mito',
 	'GenBank_hidden',
 	'Hidden_subtree',
 	'Comments'
]

names = pd.read_table(
    'refseq_proteomes/taxonomy/names.dmp',
    sep = sep,
    header = None,
    engine ='python'
)

names.columns = [
	'Taxid',
	'Name',
	'Unique',
	'Class'
]


Сохраним все идентификаторы классов:

In [ ]:
classes_ids = nodes[nodes['Rank'] == 'class']['Taxid'].to_list()

Также нам понадобится ранжированный по полноте (столбец 5 в нашем случае) файл с показателями BUSCO:

In [ ]:
busco = pd.read_table(
    'refseq_proteomes/busco_scores.tsv',
    header = None,
    sep = '\t'
)

busco.columns = [
    'GCF',
    'Name',
    'Taxid',
    'Lineage',
    'Count',
    'Score',
    'Single',
    'Dupl',
    'Fragm',
    'Miss'
]
busco = busco.dropna()
busco = busco.sort_values(by = ['Score'], ascending = False)

Используем столбец с идентификаторами таксонов как список:

In [ ]:
def find_class(current_taxid):
    if not (current_taxid in classes_ids):
        current_taxid = nodes[nodes['Taxid'] == current_taxid]['Parent'].to_list()
        if len(set(current_taxid)) > 1:
            raise Exception('Taxid', current_taxid, 'has multiple parents')
        elif len(set(current_taxid)) == 0:
            raise Exception('Warning! Root-tracing failed')
        elif current_taxid[0] == 1:
            return 0
        else:
            return find_class(current_taxid[0])
    else:
        return current_taxid

busco['Class'] = busco['Taxid'].map(find_class).astype(int)
print(len(busco['Class'].unique()))

В нашем случае было 36 классов, поэтому мы будем строить базу данных с как максимум 3*36=111 протеомами. Нули -- это организмы без соответствующего им класса, согласно nodes.dmp. Посмотрим на них.

In [ ]:
busco[busco['Class'] == 0]

В научной работе использовались не классы, а другие ранги для работы с этими видами. В данном случае просто возьмём в базу данных 3 протеома с нулями, а также протеомы Latimeria chalumnae, Protopterus annectens и человека.

In [ ]:
proteomes_list = [
    busco[busco['Taxid'] == 7897]['GCF'].to_list()[0],
    busco[busco['Taxid'] == 7888]['GCF'].to_list()[0],
    busco[busco['Taxid'] == 9606]['GCF'].to_list()[0]
]
# Верхние три с 0 будут добавлены позднее

Возьмём по три наиболее полных белка в каждом классе::

In [ ]:
for name, class_df in busco.groupby('Class'):
    class_df = class_df.sort_values(by = ['Score'], ascending = False)
    class_proteomes = list()
    taxids = set()
    for index, row in class_df.iterrows():
        if not (row['Taxid'] in taxids):
            class_proteomes.append(row['GCF'])
        taxids.add(row['Taxid'])
    proteomes_list += class_proteomes[:3]

Запишем идентификаторы полученных сборок.

In [ ]:
with open('./refseq_proteomes/assembly_ids.txt', 'w') as out:
    out.write('\n'.join(list(set(proteomes_list))))

#### 3. Загрузка протеомов, создание и конфигурация базы данных BLAST.

Для загрузки протеомов можно было бы использовать инструменты NCBI Datasets, но удобнее использовать опцию пакетного ввода.
- Перейдите на https://www.ncbi.nlm.nih.gov/sites/batchentrez;
- Выберите Assembly Database;
- Выберите файл "assembly_ids.txt";
- Нажмите "Retrieve";
- Нажмите "Retrieve records for ... UID(s)";
- Нажмите "Download Assemblies", выберите "RefSeq database" и "Protein FASTA (.faa)" тип файла;
- Создайте папку "./refseq_proteomes/assembly_files" и поместите туда TAR файлы;
- Нажмите "Download Assemblies", выберите "RefSeq database" и "Feature table (.txt)" тип файла;
- Поместите TAR файлы в созданную папку "./refseq_proteomes/assembly_files".

Объединим некоторые файлы.

In [ ]:
%%bash

cd ./refseq_proteomes/assembly_files
tar -xf genome_assemblies_features.tar
tar -xf genome_assemblies_prot_fasta.tar
mkdir ../busco_refseq

# Создайте файл FASTA
> ../busco_refseq/busco_refseq.fasta
for fasta_file in ./*/*.faa.gz
do
  zcat $fasta_file >> ../busco_refseq/busco_refseq.fasta
done

# Заголовки
for ft_file in ./*/*.txt.gz
do
  zcat $ft_file | head -n 1 | awk '{ \
      print $3,$11,$15,$16; \
    }' FS='\t' OFS='\t' \
    > ../busco_refseq/busco_feature_table.txt
  break
done

# Feature table
for ft_file in ./*/*.txt.gz
do
  zcat $ft_file | awk '{ \
      if ($1 == "CDS" && $2 == "with_protein") \
        print $3,$11,$15,$16; \
    }' FS='\t' OFS='\t' \
    >> ../busco_refseq/busco_feature_table.txt
done

Проверка совместимости FASTA файлов и feature table...

In [ ]:
import os
import gzip
from glob import glob

ft = pd.read_csv('refseq_proteomes/busco_refseq/busco_feature_table.txt', sep = '\t')

assemblies = set()
for path in glob('refseq_proteomes/assembly_files/*/*.faa.gz'):
    filename = os.path.basename(path)
    assembly = '_'.join(filename.split('_')[0:2])
    assemblies.add(assembly)

if assemblies == set(ft['assembly']) == set(proteomes_list):
    print("Assemblies list: ok")
else:
    raise Exception("FASTA file and assembly list do not match")

for path in glob('refseq_proteomes/assembly_files/*/*.faa.gz'):
    filename = os.path.basename(path)
    with gzip.open(path, 'r') as inp:
        proteome = inp.readlines()
    proteome = [p.decode('utf-8').split()[0][1:] for p in proteome if p.startswith(b'>')]
    assembly = '_'.join(filename.split('_')[0:2])
    curr_df = ft[ft['assembly'] == assembly]
    if set(proteome) == set(curr_df['product_accession']):
        print(assembly + ': ok')
    else:
        raise Exception(assembly + ': protein accession numbers from the feature table and FASTA files do not match')

Добавляем таксономическую информацию.

In [ ]:
busco = busco[['GCF', 'Taxid']]

taxid_dict = dict()
for index, b in busco.iterrows():
    taxid_dict[b['GCF']] = b['Taxid']

ft['#tax_id'] = ft['assembly'].map(taxid_dict)

if len(ft['#tax_id'].unique()) != len(ft['assembly'].unique()):
    raise Exception('Check the tax_id and assembly correspondence')

g2r = ft

g2r = g2r.rename(columns = {
    'product_accession': 'protein_accession.version',
    'symbol': 'Symbol'
})

g2r.to_csv('refseq_proteomes/g2r.tsv', sep = '\t', index = False)

Наконец, давайте создадим базу данных BLAST search (при необходимости укажите путь к вашему двоичному файлу makeblastdb).

In [ ]:
%%bash

cd ./refseq_proteomes/busco_refseq
makeblastdb \
  -dbtype prot \
  -in ./busco_refseq.fasta \
  -title busco_refseq \
  -parse_seqids \
  -out busco_refseq \
  -max_file_sz 2GB

Отредактируйте файл .ncbirc (вставьте необходимые пути).

In [ ]:
%%bash

busco_refseq_path=$(readlink -f ./refseq_proteomes/busco_refseq)
echo "[BLAST]" > ~/.ncbirc
echo "BLASTDB=${busco_refseq_path}" >> ~/.ncbirc
echo "DATA_LOADERS=blastdb" >> ~/.ncbirc
echo "BLASTDB_PROT_DATA_LOADER=busco_refseq" >> ~/.ncbirc

Отредактируйте файл конфигурации (при необходимости замените blastdbcmd_path и blastp_path).

In [ ]:
%%bash

refseq_proteomes_path=$(readlink -f ./refseq_proteomes)
blastdbcmd_path=$(which blastdbcmd)
blastp_path=$(which blastp)
echo "# Local 'gene2refseq' file" > ./cogconf.txt
echo "path2G2R:${refseq_proteomes_path}/g2r.tsv" >> ./cogconf.txt
echo "# If you use refseq, download this file and unzip, then provide names.dmp file: ftp://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump.tar.gz" >> ./cogconf.txt
echo "path2T2N:${refseq_proteomes_path}/taxonomy/names.dmp" >> ./cogconf.txt
echo "# Name of database with representative taxids" >> ./cogconf.txt
echo "databaseName:busco_refseq" >> ./cogconf.txt
echo "# Path to Blastp utility" >> ./cogconf.txt
echo "path2blastp:${blastp_path}" >> ./cogconf.txt
echo "# Path to BlastDBCmd utility" >> ./cogconf.txt
echo "blastdbcmd:${blastdbcmd_path}" >> ./cogconf.txt

Ваша база данных готова к работе! Пожалуйста, создайте среду conda для этого инструмента:
```
conda env create -n [NEW_ENV_NAME] -f dependencies.yml
```
...активируйте её...
```
conda activate [NEW_ENV_NAME]
```
 и протестируйте свою установку (по умолчанию алгоритм использует 40 потоков; введите свое значение в параметре "-t" - в этом примере мы используем 10 потоков):

In [ ]:
%%bash

echo "NP_001104262" > ./refseq_proteomes/test_input.txt
# Опционально: раскомментируйте следующую строку, чтобы последовательно проанализировать два гена.
# echo "NP_001278" >> ./refseq_proteomes/test_input.txt

# Проанализируйте test_input.txt

python3 ./cog.py \
    ./refseq_proteomes/test_input.txt \
    ./refseq_proteomes/test_output \
    -t 10